In [ ]:
%pip install gensim==4.1.2

# 这是一个词向量训练库
# 本来打算分词的，但后来因为数据集过小，
# 所以我取消了这个的想法
# （也就是说 gensim 没有实质性作用）
# 但这个程序依赖这个库，我也懒得改了

In [ ]:
# 这是训练的部分，需要GPU
# 如果只是使用，就不用看这里了
# 你首先要准备类似 拉康黑话数据集.txt 一样的文本
# 才能准备 words.model 那样的 gensim 词向量文件
# words.model 的创建看下一格
# 由于训练时数据会全部加载入内存中，所以请保证CPU内存充足

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from dataLoad import TextData
from torch import optim
from net import myModle
from matplotlib import pyplot as plt

LR = 0.003
BS = 32

data = TextData("拉康黑话数据集.txt","words.model", 10)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
network = myModle(words_num=data.dict_len, hidden_size=50).to(device)

loader = DataLoader(dataset=data,batch_size=BS,shuffle=True)
optimizer = optim.Adam(params=network.parameters(),lr=LR)
loss = nn.CrossEntropyLoss().to(device)


drawY = np.array([])

for epoch in tqdm(range(200)):
    for i, data in enumerate(loader):
        data = data.to(device)
        data = data.to(torch.long)
        index = data[:,1:] # (句数, 句中第几个词？) 每个单元装一个索引
        inp = network(data)[:, :-1, :] # 输出是一个概率分布 (句数, 句中第几个词？ 词表)
        inp = torch.transpose(inp, 2, 1)  # (句数, 词表, 句中第几个词？)
        l = loss(inp, index)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

    drawY = np.append(drawY, l.item())
plt.plot(np.arange(len(drawY)),drawY)
plt.show()

torch.save(network, '拉康快乐机.pth')

In [6]:
# words.model 的创建，如果只是使用，就不用看这里了

import multiprocessing
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models.word2vec import PathLineSentences

input_dir = input("请输入语料文件：")
f = open(input_dir, "r",encoding="utf-8")
text = f.read()
f.close()
model = Word2Vec(text, vector_size=50, window=10, min_count=2,
                 workers=multiprocessing.cpu_count(), epochs=100)

model.save("words.model")


In [ ]:
# 运行本单元格，开始成为精神分析大师！
# 确实有新的句子，但严重过拟合……
# 没什么好说的，它需要更多数据！
import using
using.eval()